In [1]:
from lasertram import LaserTRAM, LaserCalc
import lasertram as lt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm


In [2]:
data = pd.read_excel(r"C:\Users\jlubbers\OneDrive - DOI\Research\Coding\laserTRAM-DB\tests\spot_test_timestamp_raw_data.xlsx").set_index('SampleLabel')
data.head()

,timestamp,Time,7Li,24Mg,27Al,29Si,43Ca,48Ti,57Fe,88Sr,138Ba,139La,140Ce,153Eu,208Pb
SampleLabel,,,,,,,,,,,,,,,
BCR-2G_1,2021-03-01 22:08:14,13.24,500.010000,0.0,100.0004,46285.535670,100.0004,0.0000,600.014400,0.0,0.0,0.0,0.0,0.0,100.0004
BCR-2G_1,2021-03-01 22:08:14,163.60,700.019601,0.0,200.0016,43274.778818,0.0000,200.0016,900.032401,0.0,0.0,0.0,0.0,0.0,0.0000
BCR-2G_1,2021-03-01 22:08:14,313.96,600.014400,0.0,400.0064,46586.651171,300.0036,200.0016,1000.040002,0.0,0.0,0.0,0.0,0.0,200.0016
BCR-2G_1,2021-03-01 22:08:14,464.23,800.025601,0.0,100.0004,45683.326387,100.0004,100.0004,1400.078404,0.0,0.0,0.0,0.0,0.0,0.0000
BCR-2G_1,2021-03-01 22:08:14,614.50,600.014400,0.0,300.0036,46185.165445,200.0016,200.0016,700.019601,0.0,0.0,0.0,0.0,0.0,0.0000


In [3]:
samples = data.index.unique().dropna().tolist()

sample = samples[0]

bkgd_interval = (5,10)
keep_interval = (20,50)

spot = LaserTRAM(name = sample)
spot.get_data(data.loc[sample,:])
spot.assign_int_std("29Si")
spot.assign_intervals(bkgd_interval, keep_interval)
spot.get_bkgd_data()
spot.subtract_bkgd()
spot.get_detection_limits()
spot.subtract_bkgd()
spot.normalize_interval()
spot.make_output_report()

In [4]:
spot = LaserTRAM(name = sample)
lt.process_spot(
        spot,
        raw_data = data.loc[sample,:],
        bkgd = (5,10),
        keep = (20,50),
        internal_std = "29Si",
        despike = False,
        output_report = True
        )

In [5]:
bkgd_interval = (5,8)
keep_interval = (25,45)

int_std = "29Si"

my_spots = []
print("PROCESSING SPOTS\n")
for sample in tqdm(samples):
    spot = LaserTRAM(name = sample)
    lt.process_spot(
        spot,
        raw_data = data.loc[sample,:],
        bkgd = bkgd_interval,
        keep = keep_interval,
        internal_std = int_std,
        despike = False,
        output_report = True
        )
    my_spots.append(spot)

print("COMBINING OUTPUT REPORTS\n")
processed_df = pd.DataFrame()
for spot in tqdm(my_spots):
    processed_df = pd.concat([processed_df,spot.output_report])


PROCESSING SPOTS



  0%|          | 0/40 [00:00<?, ?it/s]

COMBINING OUTPUT REPORTS



  0%|          | 0/40 [00:00<?, ?it/s]

In [6]:
srm_data = pd.read_excel(r"C:\Users\jlubbers\OneDrive - DOI\Research\Coding\laserTRAM-DB\tests\laicpms_stds_tidy.xlsx")


In [8]:
concentrations = LaserCalc(name="experiment_test")
concentrations.get_SRM_comps(srm_data)
concentrations.get_data(processed_df)
concentrations.set_calibration_standard("BCR-2G")
concentrations.drift_check()
concentrations.get_calibration_std_ratios()
concentrations.set_internal_standard_concentrations(
    concentrations.data["Spot"],
    np.full(concentrations.data["Spot"].shape[0], 71.9),
    np.full(concentrations.data["Spot"].shape[0], 1),
)
concentrations.calculate_concentrations(SRM = True)
# concentrations.calculate_concentrations(SRM = False)
